<a href="https://colab.research.google.com/github/dj311/uob-summer-project/blob/master/code/exploring_juliet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import xml.etree.ElementTree as ElementTree

In [2]:
#!git clone https://github.com/dj311/uob-summer-project.git

In [2]:
tree = ElementTree.parse('../data/juliet-dataset-c-and-cpp/manifest-2019-06-27-05-59-16-6CSdPs.xml')

In [3]:
root = tree.getroot()

In [4]:
def is_shared_file(filename):
    return filename.startswith("shared")

In [6]:
import pandas as pd
juliet= pd.DataFrame(columns = ['testcase_ID', 'filename', 'code', 'flaw', 'flaw_loc'])

base_directory = '../data/juliet-dataset-c-and-cpp/testcases/'

for testcase in root:
    files = [
        element for element in testcase 
        if element.tag == 'file' and not is_shared_file(element.attrib.get('path'))
    ]
    for file in files:
        for mixed in file:
            flaw = mixed.attrib['name'].split(":")[0]
            flaw_loc = mixed.attrib['line']
            
    for file in files:            
        new_row = {
          "testcase_ID": testcase.attrib['id'],
          "filename": file.attrib['path'],
          "code": None,
          "flaw": flaw,
          "flaw_loc": flaw_loc,
        }
        
        with open(base_directory + file.attrib['path']) as code_file:
            new_row['code'] = code_file.read()

        juliet = juliet.append(new_row, ignore_index= True)

FileNotFoundError: [Errno 2] No such file or directory: '../data/juliet-dataset-c-and-cpp/testcases/000/061/940/CWE114_Process_Control__w32_char_connect_socket_01.c'

In [5]:
juliet.tail(5)

NameError: name 'juliet' is not defined

# Splitting Scripts into Good and Bad

In [13]:
juliet.testcase_ID = juliet.testcase_ID.astype(int)

In [15]:
print(juliet.iloc[0].code)

/* TEMPLATE GENERATED TESTCASE FILE
Filename: CWE114_Process_Control__w32_char_connect_socket_01.c
Label Definition File: CWE114_Process_Control__w32.label.xml
Template File: sources-sink-01.tmpl.c
*/
/*
 * @description
 * CWE: 114 Process Control
 * BadSource: connect_socket Read data using a connect socket (client side)
 * GoodSource: Hard code the full pathname to the library
 * Sink:
 *    BadSink : Load a dynamic link library
 * Flow Variant: 01 Baseline
 *
 * */

#include "std_testcase.h"

#include <wchar.h>

#ifdef _WIN32
#include <winsock2.h>
#include <windows.h>
#include <direct.h>
#pragma comment(lib, "ws2_32") /* include ws2_32.lib when linking */
#define CLOSE_SOCKET closesocket
#else /* NOT _WIN32 */
#include <sys/types.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <unistd.h>
#define INVALID_SOCKET -1
#define SOCKET_ERROR -1
#define CLOSE_SOCKET close
#define SOCKET int
#endif

#define TCP_PORT 27015
#define IP_ADDRESS "127.0.0.1"


#if

If bad: we pretend OMITGOOD is defined, so we delete what is in #ifndef OMITGOOD ... #endif /* OMITGOOD *
If good: we pretend OMITBAD is defined, so we delete what is in #ifndef OMITBAD ... #endif /* OMITBAD */

In [16]:
bad_df = juliet.copy()
good_df = juliet.copy()

In [17]:
def extract_good(text):
    '''Extract good script from code snippet'''
    
    text = text.split('\n')
    output_txt = []
    
    in_OMITBAD_block = False
        
    for line in text:
        if line == '#ifndef OMITBAD':
            in_OMITBAD_block = True
                
        if not in_OMITBAD_block:
            output_txt.append(line)
                
        if line == '#endif /* OMITBAD */':
            in_OMITBAD_block = False
    
    return '\n'.join(output_txt)

def extract_bad(text):
    '''Extract bad script from code snippet'''
    
    text = text.split('\n')
    output_txt = []
    
    in_OMITGOOD_block = False
        
    for line in text:
        if line == '#ifndef OMITGOOD':
            in_OMITGOOD_block = True
                
        if not in_OMITGOOD_block:
            output_txt.append(line)

        if line == '#endif /* OMITGOOD */':
            in_OMITGOOD_block = False
    
    return '\n'.join(output_txt)

In [18]:
print(extract_good(good_df.iloc[0].code))

/* TEMPLATE GENERATED TESTCASE FILE
Filename: CWE114_Process_Control__w32_char_connect_socket_01.c
Label Definition File: CWE114_Process_Control__w32.label.xml
Template File: sources-sink-01.tmpl.c
*/
/*
 * @description
 * CWE: 114 Process Control
 * BadSource: connect_socket Read data using a connect socket (client side)
 * GoodSource: Hard code the full pathname to the library
 * Sink:
 *    BadSink : Load a dynamic link library
 * Flow Variant: 01 Baseline
 *
 * */

#include "std_testcase.h"

#include <wchar.h>

#ifdef _WIN32
#include <winsock2.h>
#include <windows.h>
#include <direct.h>
#pragma comment(lib, "ws2_32") /* include ws2_32.lib when linking */
#define CLOSE_SOCKET closesocket
#else /* NOT _WIN32 */
#include <sys/types.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <unistd.h>
#define INVALID_SOCKET -1
#define SOCKET_ERROR -1
#define CLOSE_SOCKET close
#define SOCKET int
#endif

#define TCP_PORT 27015
#define IP_ADDRESS "127.0.0.1"



#i

In [19]:
print(extract_bad(good_df.iloc[0].code))

/* TEMPLATE GENERATED TESTCASE FILE
Filename: CWE114_Process_Control__w32_char_connect_socket_01.c
Label Definition File: CWE114_Process_Control__w32.label.xml
Template File: sources-sink-01.tmpl.c
*/
/*
 * @description
 * CWE: 114 Process Control
 * BadSource: connect_socket Read data using a connect socket (client side)
 * GoodSource: Hard code the full pathname to the library
 * Sink:
 *    BadSink : Load a dynamic link library
 * Flow Variant: 01 Baseline
 *
 * */

#include "std_testcase.h"

#include <wchar.h>

#ifdef _WIN32
#include <winsock2.h>
#include <windows.h>
#include <direct.h>
#pragma comment(lib, "ws2_32") /* include ws2_32.lib when linking */
#define CLOSE_SOCKET closesocket
#else /* NOT _WIN32 */
#include <sys/types.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <unistd.h>
#define INVALID_SOCKET -1
#define SOCKET_ERROR -1
#define CLOSE_SOCKET close
#define SOCKET int
#endif

#define TCP_PORT 27015
#define IP_ADDRESS "127.0.0.1"


#if

In [20]:
from preprocess_code import process_for_graph2vec

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [21]:
good_example = good_df.iloc[0]
good_example.code = extract_good(good_example.code)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [22]:
good_df.code = good_df.code.apply(extract_good)
bad_df.code = bad_df.code.apply(extract_bad)

In [23]:
#good_df.testcase_ID = good_df.testcase_ID.apply(lambda x: x)
bad_df.testcase_ID = bad_df.testcase_ID.apply(lambda x: -x)

In [24]:
print(good_df.iloc[0].code)

/* TEMPLATE GENERATED TESTCASE FILE
Filename: CWE114_Process_Control__w32_char_connect_socket_01.c
Label Definition File: CWE114_Process_Control__w32.label.xml
Template File: sources-sink-01.tmpl.c
*/
/*
 * @description
 * CWE: 114 Process Control
 * BadSource: connect_socket Read data using a connect socket (client side)
 * GoodSource: Hard code the full pathname to the library
 * Sink:
 *    BadSink : Load a dynamic link library
 * Flow Variant: 01 Baseline
 *
 * */

#include "std_testcase.h"

#include <wchar.h>

#ifdef _WIN32
#include <winsock2.h>
#include <windows.h>
#include <direct.h>
#pragma comment(lib, "ws2_32") /* include ws2_32.lib when linking */
#define CLOSE_SOCKET closesocket
#else /* NOT _WIN32 */
#include <sys/types.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <unistd.h>
#define INVALID_SOCKET -1
#define SOCKET_ERROR -1
#define CLOSE_SOCKET close
#define SOCKET int
#endif

#define TCP_PORT 27015
#define IP_ADDRESS "127.0.0.1"



#i

In [25]:
good_df['bug'] = 'False'
bad_df['bug'] = 'True'

In [26]:
juliet_split = good_df.append(bad_df)
juliet_split

,testcase_ID,filename,code,flaw,flaw_loc,bug
0,61940,000/061/940/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,121,False
1,61941,000/061/941/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,124,False
2,61942,000/061/942/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,124,False
3,61943,000/061/943/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,131,False
4,61944,000/061/944/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,131,False
5,61945,000/061/945/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,128,False
6,61946,000/061/946/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,130,False
7,61947,000/061/947/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,138,False
8,61948,000/061/948/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,124,False
9,61949,000/061/949/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,124,False


In [27]:
juliet_split.to_csv('../data/juliet_split.csv.gz')